In [1]:
import sys
sys.path.append('..')

import lcatools
import os

In [2]:
import xlrd
import re

In [3]:
from collections import defaultdict

In [4]:
lcia_file = '/data/LCI/LCIA implementation v3.1 2014_08_13.xlsx'
spold_archive = '/data/LCI/Ecoinvent/3.2/cutoff/datasets'

catalog_dir = os.path.join(os.path.expanduser('~'), 'GitHub', 'lca-tools-datafiles', 'catalogs')
spold_catfile = 'ei3.2_cutoff_spold.gz'

In [5]:
%time X = xlrd.open_workbook(lcia_file)

CPU times: user 19.2 s, sys: 20 ms, total: 19.2 s
Wall time: 19.2 s


In [10]:
s = X.sheet_by_index(1)

In [11]:
s.name

'impact methods'

In [12]:
b = X.sheet_by_name('impact methods')

In [62]:
headings

['method',
 'category',
 'indicator',
 'name',
 'compartment',
 'subcompartment',
 'CF 3.01',
 'CF 3.1',
 'Change?',
 'note',
 'Known issue']

In [46]:
headings[0].value

'method'

In [52]:
g = b.get_rows()
headings = [h.value for h in next(g)]

In [53]:
headings[0]

'method'

In [54]:
cfs = []
for row in g:
    d = dict()
    for i,h in enumerate(headings):
        d[h] = row[i]
    cfs.append(d)

In [55]:
len(cfs)

170915

In [56]:
cfs[0]

{'CF 3.01': number:1.6,
 'CF 3.1': number:1.6,
 'Change?': text:'FALSE',
 'Known issue': empty:'',
 'category': text:'acidification potential',
 'compartment': text:'air',
 'indicator': text:'average European',
 'method': text:'CML 2001',
 'name': text:'Ammonia',
 'note': text:'',
 'subcompartment': text:'low population density, long-term'}

In [5]:
from lcatools.providers.ecoinvent_lcia import EcoinventLcia
from lcatools.db_catalog import from_json
from lcatools.providers.ecospold2 import EcospoldV2Archive

In [6]:
lcia_file

'/data/LCI/LCIA implementation v3.1 2014_08_13.xlsx'

es_upstream = EcospoldV2Archive('/data/LCI/Ecoinvent/3.2/cutoff/datasets', quiet=True)

In [7]:
es_upstream = from_json(os.path.join(catalog_dir, spold_catfile))

Path points to a directory. Assuming expanded archive
19 new quantity entities added (19 total)
4681 new flow entities added (4681 total)
11578 new process entities added (11578 total)


In [8]:
[k['referenceUnit'].unitstring() for k in es_upstream.quantities()]

['person*km',
 'm*year',
 'm3',
 'unit',
 'ha',
 'kWh',
 'metric ton*km',
 'km*year',
 'm2*year',
 'm3*year',
 'kBq',
 'm',
 'hour',
 'km',
 'l',
 'kg',
 'm2',
 'MJ',
 'kg*day']

In [9]:
EL = EcoinventLcia(lcia_file, upstream=es_upstream, mass_quantity=es_upstream.quantity_with_unit('kg'), 
                   ns_uuid='46802ca5-8b25-398c-af10-2376adaa4623')

Adding quantity entity with 487df68b-4994-4027-8fdc-a4dc298257b7: EcoSpold Quantity kg


In [10]:
%time EL._load_xl_rows()

CPU times: user 22 s, sys: 70 ms, total: 22 s
Wall time: 22.1 s


In [11]:
len(EL._xl_rows)

170915

In [12]:
for row in EL._xl_rows[:10]:
    EL._create_flow(row)

Found upstream match: Ammonia (CAS 007664-41-7) [air, low population density, long-term]
Adding flow entity with 825de299-9656-3d57-a6d6-e04093ff21f5: Ammonia
Adding flow entity with 30f6b7f3-65b0-3705-b096-a2ef311315c2: Ammonia
Found upstream match: Ammonia (CAS 007664-41-7) [air, non-urban air or from high stacks]
Adding flow entity with becd3615-9110-3e27-be55-e1e6261276bd: Ammonia
Found upstream match: Ammonia (CAS 007664-41-7) [air, unspecified]
Adding flow entity with 161e309c-49de-3936-a1c4-7ffd9de33f47: Ammonia
Found upstream match: Ammonia (CAS 007664-41-7) [air, urban air close to ground]
Adding flow entity with fc10a7a9-5532-3e60-8b6b-b4398f11bf2a: Ammonia
Found upstream match: Nitrogen oxides (CAS 011104-93-1) [air, low population density, long-term]
Adding flow entity with 88190311-886f-3106-af98-bee5edb80b8c: Nitrogen oxides
Found upstream match: Nitrogen oxides (CAS 011104-93-1) [air, lower stratosphere + upper troposphere]
Adding flow entity with a2ba07ea-fb23-310a-8e5a

In [13]:
f = EL.search('30f6b7f3-65b0-3705-b096-a2ef311315c2')[0]

In [14]:
str(f)

'Ammonia [air, lower stratosphere + upper troposphere]'

In [15]:
es_upstream.search(entity_type='flow', Name='Ammonia', Compartment='tropo')

[]

In [16]:
EL.search(entity_type='flow', Name='Ammonia', Compartment='tropo')

In [17]:
EL.flows()[0].__dict__

{'_d': {'CasNumber': '011104-93-1',
  'Comment': 'EcoSpold02 Flow',
  'Compartment': ['air', 'lower stratosphere + upper troposphere'],
  'Name': 'Nitrogen oxides',
  'origin': '/data/LCI/Ecoinvent/3.2/cutoff/datasets'},
 '_external_ref': 'Nitrogen oxides, air, lower stratosphere + upper troposphere',
 '_ref_quantity_factor': 1,
 '_uuid': UUID('4841a0fe-c250-4a39-94a1-1bc31426c0f1'),
 'entity_type': 'flow',
 'reference_entity': <lcatools.entities.LcQuantity at 0x7fe837385f60>}

In [18]:
EL.flows()[0].get_signature()

{'CasNumber': '011104-93-1',
 'Comment': 'EcoSpold02 Flow',
 'Compartment': ['air', 'lower stratosphere + upper troposphere'],
 'EntityType': 'flow',
 'Name': 'Nitrogen oxides',
 'referenceQuantity': <lcatools.entities.LcQuantity at 0x7fe837385f60>}

In [19]:
EL._quiet=True

In [20]:
%time EL.load_all()

0 new process entities added (0 total)
3255 new flow entities added (3255 total)
702 new quantity entities added (702 total)
CPU times: user 10.7 s, sys: 90 ms, total: 10.8 s
Wall time: 10.8 s


In [21]:
len(EL.characterizations())

170755

In [23]:
len([i for i in EL._characterizations.zeros()])

160

In [24]:
EL.characterizations()[0].quantity.get_uuid()

'0755d155-caab-3ecf-9eaf-46d1d595cf8d'

In [52]:
def flows_gen():
    for _f in EL.flows():
        yield _f

In [36]:
def en_gen(archive):
    for k in archive._entities.keys():
        v = archive._get_entity(k)
        if v.entity_type == 'flow':
            yield v
u = en_gen(es_upstream)

In [37]:
e = next(u)

In [38]:
next(u)['Compartment']

['natural resource', 'land']

In [46]:
e = next(u)
[e['Name']] + e['Compartment']

['Cesium-134', 'water', 'surface water']

In [35]:
e

In [31]:
x

In [53]:
flows = flows_gen()

In [69]:
%timeit -n3 print(len(EL._characterizations.get(flow=next(flows))))

68
177
38
23
11
5
68
63
4
3 loops, best of 3: 56.3 ms per loop


In [54]:
f = next(flows)

In [55]:
f

In [66]:
len(EL._characterizations.get(flow=next(flows)))

1

In [61]:
f=next(flows)
f.get_signature()

{'CasNumber': '',
 'Comment': '',
 'Compartment': ['air', 'non-urban air or from high stacks'],
 'EntityType': 'flow',
 'Name': 'Dipropylamine',
 'referenceQuantity': <lcatools.entities.LcQuantity at 0x7fb483ee9828>}

In [51]:
len([f for f in flows()])

3255

In [71]:
cf = EL._characterizations.get(flow=next(flows))

In [72]:
len(cf)

152

In [75]:
cf[0].__dict__

{'flow': <lcatools.entities.LcFlow at 0x7fb44b5dd710>,
 'quantity': <lcatools.entities.LcQuantity at 0x7fb483ed0470>,
 'value': 224400.0}

In [79]:
def cf_str(cf):
    return { 'flow': str(cf.flow), 'quantity': str(cf.quantity), 'value': cf.value}

In [86]:
cf_str(cf[2])

{'flow': 'Cadmium, ion [water, ground-]',
 'quantity': 'LC quantity: ReCiPe Endpoint (H,A) w/o LT, ecosystem quality w/o LT, freshwater ecotoxicity w/o LT',
 'value': 0.0172175497759}

In [90]:
cf[2].value.keys()

dict_keys(['category', 'indicator', 'method', 'name', 'note', 'CF 3.1', 'CF 3.01', 'subcompartment', 'Known issue', 'compartment'])

In [103]:
cf[3].value['indicator']

'carcinogenics'

In [38]:
unique_names = set([f['Name'] for f in EL.flows()])

In [39]:
len(unique_names)

984

In [40]:
list(unique_names)[:10]

['Mercury',
 'Benzene, chloro-',
 'Occupation, pasture, man made',
 'Tolylfluanid',
 'Transformation, from dump site, inert material landfill',
 'Monochloroethane',
 'Pd, Pd 7.3E-4%, Pt 2.5E-4%, Rh 2.0E-5%, Ni 2.3E+0%, Cu 3.2E+0% in ore, in ground',
 'Ammonium carbonate',
 'Oxadixyl',
 '3-Methyl-1-butanol']

In [28]:
f = EL.flows()[123]
q = EL.quantities()[698]

In [36]:
%timeit -n5 print('%d' % len([i for i in EL.characterizations() if i.quantity is q]))

260
260
260
260
260
260
260
260
260
260
260
260
260
260
260
5 loops, best of 3: 147 ms per loop


In [37]:
%timeit -n5 print('%d' % len(EL._characterizations.get(quantity=q)))

260
260
260
260
260
260
260
260
260
260
260
260
260
260
260
5 loops, best of 3: 57.6 ms per loop


In [24]:
EL.quantities()[0]['referenceUnit'].unitstring()

'points'

In [18]:
EL._xl_rows[0]

{'CF 3.01': 1.6,
 'CF 3.1': 1.6,
 'Change?': 'FALSE',
 'Known issue': '',
 'category': 'acidification potential',
 'compartment': 'air',
 'indicator': 'average European',
 'method': 'CML 2001',
 'name': 'Ammonia',
 'note': '',
 'subcompartment': 'low population density, long-term'}

In [17]:
f = EL.flows()[0]

In [22]:
EL.load_all()

0 new process entities added (0 total)
3245 new flow entities added (3255 total)
0 new quantity entities added (702 total)


In [32]:
cf = EL.characterizations()[100000]

In [33]:
cf.__dict__

{'flow': <lcatools.entities.LcFlow at 0x7fa002887f60>,
 'quantity': <lcatools.entities.LcQuantity at 0x7fa009760208>,
 'value': None}

In [26]:
EL.write_to_file(os.path.join(catalog_dir, 'ei_lcia.gz'), gzip=True, characterizations=False)

In [25]:
EL.flows()[100].get_signature()

{'CasNumber': '',
 'Comment': '',
 'Compartment': ['water', 'ground-, long-term'],
 'EntityType': 'flow',
 'Name': 'Ethane, 1,1,1-trichloro-, HCFC-140',
 'referenceQuantity': <lcatools.entities.LcQuantity at 0x7fe837385f60>}